In [1]:
from lib import data_loaders
import requests

In [2]:
DendraClient = data_loaders.Dendra(email="dragar@lynker.com", password="DangerLynk")

successfully authenticated


In [3]:
DendraClient.get_datastreams(
    datastream_name="Well Water Level xle/lev", station_id="64dbe9ca5c0d5f0fd8694a9f"
)

'64dd3e755526452f533ea4f5'

In [ ]:
# TODO make get_datastreams internal, and return data with the datapoints endpoint